# test

## include scripts

### include notebook core

In [ ]:
. ../../../../scripts/nbs_header.ps1

### Include core functions script

In [ ]:
. ../../../../scripts/core.ps1

### Include spiral library

In [ ]:
. ../../../../deps/spiral/lib/spiral/lib.ps1

## execute project commands

### run notebook with retries using spiral supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --execute-command "../../../../deps/spiral/workspace/target/release/spiral$(_exe) dib --path test.dib --retries 3" } | Invoke-Block

00:00:00 v #1 async.run_with_timeout_async / { timeout = 180 }
00:00:01 d #1 runtime.execute_with_options_async / { file_name = ../../../../deps/spiral/workspace/target/release/spiral.exe; arguments = US5_0 "dib --path test.dib --retries 3"; options = { command = ../../../../deps/spiral/workspace/target/release/spiral.exe dib --path test.dib --retries 3; cancellation_token = Some System.Threading.CancellationToken; environment_variables = [||]; on_line = None; stdin = None; trace = true; working_directory = None } }
00:00:01 v #2 > 00:00:00 d #1 spiral.main / { args = Array(MutCell(["dib", "--path", "test.dib", "--retries", "3"])) }
00:00:01 v #3 > 00:00:00 d #2 runtime.execute_with_options / { file_name = dotnet; arguments = ["repl", "--exit-after-run", "--run", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib", "--output-path", "c:/home/git/polyglot/apps/spiral/temp/test/test.dib.ipynb"]; options = { command = dotnet repl --exit-after-run --run "c:/home/git/polyglot/apps/spiral/t

### parse the .dib file into .spi format with dibparser

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(_exe) test.dib spi } | Invoke-Block

00:00:00 d #1 writeDibCode / output: Spi / path: test.dib
00:00:00 d #2 parseDibCode / output: Spi / file: test.dib


### build .fsx file from .spi using supervisor

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(_exe) --build-file test.spi test.fsx } | Invoke-Block

00:00:00 v #1 async.run_with_timeout_async / { timeout = 180 }
00:00:01 v #2 async.run_with_timeout_async / { timeout = 180 }
00:00:01 d #1 Supervisor.buildFile / takeWhileInclusive / path: test.spi / errors: [] / typeErrorCount: 0 / retry: 0 / outputContent:

00:00:01 d #2 Supervisor.buildFile / AsyncSeq.scan / path: test.spi / errors: [] / outputContentResult:  / typeErrorCount: 0 / retry: 0 / error:  / outputContent:

00:00:01 d #3 Supervisor.buildFile / takeWhileInclusive / path: test.spi / errors: [] / typeErrorCount: 0 / retry: 0 / outputContent:

00:00:01 v #4 Supervisor.sendJson / port: 13805 / json: {"FileOpen":{"spiText":"/// # test (Polyglot)\n\n/// ## main\ninl main (_args : array_base string) =...0022\u003C/test\u003E\u0022\n","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
00:00:01 v #5 Supervisor.sendJson / port: 13805 / json: {"BuildFile":{"backend":"Fsharp","uri":"file:///c:/home/git/polyglot/apps/spiral/temp/test/test.spi"}} / result:
0

## compile and format the project

### compile project with fable targeting optimized rust

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 5.0.0-alpha.2: F# to Rust compiler (status: alpha)

Thanks to the contributor! @dbrattli
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 185ms

Started Fable compilation...

Fable compilation finished in 2147ms

.\test.fsx(11,0): (11,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


### fix formatting issues in the .rs file using regex and set-content

In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
    | FixRust `
| Set-Content test.rs

### format the rust code using cargo fmt

In [ ]:
cargo fmt --

## build and test the project

### build the project in release mode using nightly rust compiler

In [ ]:
cargo build --release

   Compiling proc-macro2 v1.0.92
   Compiling unicode-ident v1.0.14
   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling quote v1.0.37
   Compiling syn v2.0.90
   Compiling zerocopy-derive v0.7.35
   Compiling thiserror-impl v1.0.69
   Compiling zerocopy v0.7.35
   Compiling thiserror v1.0.69
   Compiling ppv-lite86 v0.2.20
   Compiling rand_chacha v0.3.1
   Compiling rand v0.8.5
   Compiling proptest v1.5.0
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 25.76s


### run release tests with output enabled

In [ ]:
{ cargo test --release -- --show-output } | Invoke-Block

   Compiling zerocopy v0.7.35
   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\lib\rust\fable\fable_modules\fable-library-rust)
   Compiling thiserror v1.0.69
   Compiling ppv-lite86 v0.2.20
   Compiling rand_chacha v0.3.1
   Compiling rand v0.8.5
   Compiling proptest v1.5.0
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished `release` profile [optimized] target(s) in 53.52s
     Running unittests main.rs (C:\home\git\polyglot\workspace\target\release\deps\spiral_temp_test-5730544e07be5619.exe)

running 3 tests
test test_parse_number ... ok
test prop_parse_format_idempotent ... ok
test adding_and_then_removing_an_item_from_the_cart_leaves_the_cart_unchanged ... ok

successes:

---- prop_parse_format_idempotent stdout ----
input=StringLiteral("d5;&eA)=L1&*@D.zMo'.$%cy`.#6$L")
input=Integer(2643217931120806427)
input=Identifier("CB5o209o57Y7Lb89SQj7hXDHtxYRfUGI")
input=Operator("-")
input=Comment("!oF#A\"}J:")
input=Integer(86547171

### execute the binary in release mode

In [ ]:
{ . $ScriptDir/../../../../workspace/target/release/spiral_temp_test$(_exe) } | Invoke-Block

app=test
